In [330]:
import time
from sage.rings.polynomial.hilbert import hilbert_poincare_series

# computes degree of regularity for a homogeneous ideal
def hdegree_reg(I):
    G = I.groebner_basis()
    LT = Ideal([g.lt() for g in G])
    HS = hilbert_poincare_series(LT)
    delta = HS.numerator().degree()
    l = HS.denominator().degree()
    return(delta - l + 1)


# computes F^top
def f_top(I, R):
    f_list = I.gens()
    new_ring.<h> = PolynomialRing(R, order = 'degrevlex')
    homogenized = [new_ring(poly.homogenize()) for poly in f_list]
    final = [R(poly.subs(h=0)) for poly in homogenized] 
    return(final)


# computes degree of regularity in the nonhomogeneous case
def degree_reg(I, R):
    J = ideal(f_top(I,R))
    return hdegree_reg(J)

  
def test_rowsp(f, f_list, d, ring):
    mons = get_mons(d, ring)
    M = mac_matrix(f_list, d, ring)
    done = False
    while not done:
        rowsp = M.row_space()
        to_be_added = []
        for row in M.rref().rows():
            g = 0
            for (coeff, mon) in zip(row, mons):
                g += coeff * mon
            max_test_deg = d - g.degree()
            if (g != 0) and (max_test_deg > 0):
                for mon in get_mons(max_test_deg, ring):
                    test_p = mon*g
                    test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                    if test_vec not in rowsp:
                        to_be_added.append(test_vec)
        if to_be_added == []:
            done = True
        else:
            new_rows = M.rows() + to_be_added
            M = Matrix(new_rows)
    f_vec = vector([ring(f).monomial_coefficient(mon) for mon in mons])
    return f_vec in M.row_space()


def get_mons(d, ring, leq = True):
    if leq:
        mons = []
        for i in range(d+1):
            mons += [ring({tuple(a):1}) for a in WeightedIntegerVectors(i,[1 for gen in ring.gens()])]
    else:
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
    mons.sort()
    mons.reverse()
    return mons


def deg_fall(f, f_list, ring):
    test_deg = f.degree()
    while True:
        print(test_deg, time.strftime("%H:%M:%S", time.localtime()))
        if test_rowsp(f, f_list, test_deg, ring):
            return test_deg
        test_deg += 1
        
        
def lfd(f_list, ring):
    fall_degrees = []
    I = ideal(f_list)
    B = I.groebner_basis()
    print(B)
    for f in B:
        print(f)
        d = deg_fall(f, f_list, ring)
        if d > f.degree():
            fall_degrees.append(d)
    if fall_degrees == []:
        return 0
    else:
        return max(fall_degrees)
    
    
# Compute the Macaulay matrix in degree d of a system f = [f1,...,fs]
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


# Takes an RREF Macaulay matrix as input and returns a list of the corresponding polynomials
def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


# Now instead of using mac_matrix followed by recover_polys, we ask for the new polynomials directly:
def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


# Now define a function that calculates a Gröbner basis and the solving degree (per Caminata and Gorla, 2021)
def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1
        

# Modified version to compute the solving degree per Caminata and Gorla, 2023
def sdeg(f, ring):
    d = 0
    while True:
        d_level_base = []
        mons = get_mons(d, ring)
        M = mac_matrix(f, d, ring)
        done = False
        while not done:
            rowsp = M.row_space()
            to_be_added = []
            for row in M.rref().rows():
                g = 0
                for (coeff, mon) in zip(row, mons):
                    g += coeff * mon
                max_test_deg = d - g.degree()
                if (g != 0) and (max_test_deg > 0):
                    for mon in get_mons(max_test_deg, ring):
                        test_p = mon*g
                        test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                        if test_vec not in rowsp:
                            to_be_added.append(test_vec)
            if to_be_added == []:
                done = True
                for row in M.rref().rows():
                    g = 0
                    for (coeff, mon) in zip(row, mons):
                        g += coeff * mon
                    if g != 0:
                        d_level_base.append(g)
                if d_level_base == []:
                    d_level_base = [ring(0)]
                check_1 = Sequence(d_level_base).is_groebner()
                check_2 = (ideal(d_level_base) == ideal(f))
                print(d, Sequence(d_level_base), check_1, check_2)
                if check_1 and check_2:
                    return (d, Sequence(d_level_base))
                d += 1
            else:
                new_rows = M.rows() + to_be_added
                M = Matrix(new_rows)

In [333]:
# tests conjecture 3 for the degree reverse lex ordering (Sage default)

def is_generic(I,R):
    r = I.dimension()
    if r == 0: 
        return True
    r_gens = list(R.gens())
    M = ideal(r_gens)
    variables_to_check = r_gens[r-2:][::-1]
    variables_checked = []
    for variable in variables_to_check:
        J = I
        for var in variables_checked:
            J = J + ideal(var)
        if J.dimension() == 0:
            return True
        sat = J.saturation(M)[0]
        for prime in sat.associated_primes():
            if variable in prime:
                return False
        variables_checked.append(variable)
    return True

def test_conjecture(f_list, R):
    F = ideal(f_list)
    S = F.homogenize().ring()
    h_list = [S(f.homogenize()) for f in f_list]
    Fh = ideal(h_list)
    reg = Fh._singular_().mres(0).betti().nrows()
    print(reg)
    sd = sdeg(f_list, R)[0]
    return (sd, reg, is_generic(Fh,S))

def test_conjecture_extra(f_list, R):
    F = ideal(f_list)
    S = F.homogenize().ring()
    h_list = [S(f.homogenize()) for f in f_list]
    Fh = ideal(h_list)
    reg = Fh._singular_().mres(0).betti().nrows()
    print(reg, '\n')
    sd = sdeg(f_list, R)[0]
    print('')
    solve_deg = mac_grob(f_list, R)[0]
    return (Fh.dimension(), sd, reg, solve_deg, is_generic(Fh,S))

In [320]:
R.<x1,x2,x3,x4> = PolynomialRing(QQ, order = 'degrevlex')
f1 = x1^2 + x2^2
f2 = x1^2*x2^2
test_conjecture([f1,f2],R)

5
0 [0] True False
1 [0] True False
2 [x1^2 + x2^2] True False
3 [x1^3 + x1*x2^2, x1^2*x2 + x2^3, x1^2*x3 + x2^2*x3, x1^2*x4 + x2^2*x4, x1^2 + x2^2] True False
4 [x1^4, x1^3*x2 + x1*x2^3, x1^2*x2^2, x2^4, x1^3*x3 + x1*x2^2*x3, x1^2*x2*x3 + x2^3*x3, x1^2*x3^2 + x2^2*x3^2, x1^3*x4 + x1*x2^2*x4, x1^2*x2*x4 + x2^3*x4, x1^2*x3*x4 + x2^2*x3*x4, x1^2*x4^2 + x2^2*x4^2, x1^3 + x1*x2^2, x1^2*x2 + x2^3, x1^2*x3 + x2^2*x3, x1^2*x4 + x2^2*x4, x1^2 + x2^2] True True


(4, 5, True)

In [321]:
# Gorla 4.1

R.<x,y> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^10 - 1
test_conjecture([f1,f2,f3],R)

11
0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2 - 1, x*y + y, y^2 - 1, x + 1] True True


(3, 11, True)

In [323]:
# Gorla 4.2
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^6 - 1
test_conjecture([f1,f2,f3],R)

8
0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2*z - z, x*y*z + y*z, y^2*z - z, x*z^2 + z^2, x^2 - 1, x*y + y, y^2 - 1, x*z + z, x + 1] True False
4 Polynomial Sequence with 26 Polynomials in 3 Variables True False
5 Polynomial Sequence with 45 Polynomials in 3 Variables True False
6 Polynomial Sequence with 72 Polynomials in 3 Variables True True


(6, 8, False)

In [324]:
# Gorla 4.3
R.<x,y> = PolynomialRing(GF(13), order = "degrevlex")
f1 = x^2 - 1
f2 = y^2 - 1
I = Ideal(f1,f2)
test_conjecture([f1,f2],R)

3
0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True


(2, 3, True)

In [352]:
# Gorla 4.4
R.<w,x,y,z> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
test_conjecture([f1,f2,f3,f4],R)

13
0 [0] True False
1 [0] True False
2 [x^2 - 1, x*y - 1] False False
3 [w^2*x - w^2*y, w*x^2 - w, x^3 - y, w*x*y - w, x^2*y - y, w*y^2 - w, x*y^2 - y, y^3 - y, w*x*z - w*y*z, x^2*z - z, x*y*z - z, y^2*z - z, x*z^2 - y*z^2, w*x - w*y, x^2 - 1, x*y - 1, y^2 - 1, x*z - y*z, x - y] True False
4 Polynomial Sequence with 45 Polynomials in 4 Variables True False
5 Polynomial Sequence with 91 Polynomials in 4 Variables True False
6 Polynomial Sequence with 165 Polynomials in 4 Variables True False
7 Polynomial Sequence with 275 Polynomials in 4 Variables True False
8 Polynomial Sequence with 431 Polynomials in 4 Variables False True
9 Polynomial Sequence with 683 Polynomials in 4 Variables True True


(9, 13, False)

In [328]:
# Gorla 2.4
R.<w,x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
# test_conjecture([f1,f2,f3,f4],R) gives (11,15,False)

In [334]:
R.<w,x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x*y^2 - z 
f2 = z^4*w + 1
test_conjecture_extra([f1,f2],R)

7 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [w^2*x*y^2 - w^2*z, w*x^2*y^2 - w*x*z, x^3*y^2 - x^2*z, w*x*y^3 - w*y*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, w*x*y^2*z - w*z^2, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, w*z^4 + 1, w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [w^2*x*y^2 - w^2*z, w*x^2*y^2 - w*x*z, x^3*y^2 - x^2*z, w*x*y^3 - w*y*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, w*x*y^2*z - w*z^2, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, w*z^4 + 1, w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True True


(3, 5, 7, 5, False)

In [335]:
R.<w,x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x*y^2 - z 
f2 = z^4*w
test_conjecture_extra([f1,f2],R)

7 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [w^2*x*y^2 - w^2*z, w*x^2*y^2 - w*x*z, x^3*y^2 - x^2*z, w*x*y^3 - w*y*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, w*x*y^2*z - w*z^2, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, w*z^4, w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [w^2*x*y^2 - w^2*z, w*x^2*y^2 - w*x*z, x^3*y^2 - x^2*z, w*x*y^3 - w*y*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, w*x*y^2*z - w*z^2, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, w*z^4, w*x*y^2 - w*z, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True True


(3, 5, 7, 5, False)

In [336]:
R.<w,x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x*y^2
f2 = z^4*w
test_conjecture_extra([f1,f2],R)

7 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True False
4 [w*x*y^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
5 [w^2*x*y^2, w*x^2*y^2, x^3*y^2, w*x*y^3, x^2*y^3, x*y^4, w*x*y^2*z, x^2*y^2*z, x*y^3*z, x*y^2*z^2, w*z^4, w*x*y^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True False
4 [w*x*y^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
5 [w^2*x*y^2, w*x^2*y^2, x^3*y^2, w*x*y^3, x^2*y^3, x*y^4, w*x*y^2*z, x^2*y^2*z, x*y^3*z, x*y^2*z^2, w*z^4, w*x*y^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True True


(2, 5, 7, 5, False)

In [342]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2
test_conjecture_extra([f1],R)

3 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True True


(2, 3, 3, 3, False)

In [344]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2
f2 = x^7 - x
f3 = y^7 - y
f4 = z^7 - z
test_conjecture_extra([f1,f2,f3,f4],R)

19 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True False
4 [x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
5 [x^3*y^2, x^2*y^3, x*y^4, x^2*y^2*z, x*y^3*z, x*y^2*z^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 38 Polynomials in 3 Variables False True
8 Polynomial Sequence with 94 Polynomials in 3 Variables True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2] True False
4 [x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
5 [x^3*y^2, x^2*y^3, x*y^4, x^2*y^2*z, x*y^3*z, x*y^2*z^2, x^2*y^2, x*y^3, x*y^2*z, x*y^2] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 38 Polynomials in 3 Variables False True
8 Polynomial Sequence with 68 Polynomials in 3 Variables True True


(1, 8, 19, 8, True)

In [345]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2 - z
test_conjecture_extra([f1],R)

3 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True True


(3, 3, 3, 3, False)

In [346]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2 - z
f2 = x^7 - x
f3 = y^7 - y
f4 = z^7 - z
test_conjecture_extra([f1,f2,f3,f4],R)

17 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 38 Polynomials in 3 Variables False True
8 Polynomial Sequence with 112 Polynomials in 3 Variables False True
9 Polynomial Sequence with 171 Polynomials in 3 Variables True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence wit

(1, 9, 17, 15, True)

In [347]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2 - z
f3 = y^7 - y
f4 = z^7 - z
test_conjecture_extra([f1,f3,f4],R)

14 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 37 Polynomials in 3 Variables False True
8 Polynomial Sequence with 109 Polynomials in 3 Variables False True
9 Polynomial Sequence with 168 Polynomials in 3 Variables True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence wit

(2, 9, 14, 15, False)

In [348]:
R.<x,y,z> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2 - z
f4 = z^7 - z
test_conjecture_extra([f1,f4],R)

9 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 36 Polynomials in 3 Variables True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z] True False
4 [x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
5 [x^3*y^2 - x^2*z, x^2*y^3 - x*y*z, x*y^4 - y^2*z, x^2*y^2*z - x*z^2, x*y^3*z - y*z^2, x*y^2*z^2 - z^3, x^2*y^2 - x*z, x*y^3 - y*z, x*y^2*z - z^2, x*y^2 - z] True False
6 Polynomial Sequence with 20 Polynomials in 3 Variables True False
7 Polynomial Sequence with 36 Polynomials in 3 Variables True True


(2, 7, 9, 7, False)

In [350]:
R.<x,y,z,h> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y^2 - z*h^2
f4 = z^7 - z*h^6
test_conjecture_extra([f1,f4],R)

9 

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z*h^2] True False
4 [x^2*y^2 - x*z*h^2, x*y^3 - y*z*h^2, x*y^2*z - z^2*h^2, x*y^2*h - z*h^3, x*y^2 - z*h^2] True False
5 [x^3*y^2 - x^2*z*h^2, x^2*y^3 - x*y*z*h^2, x*y^4 - y^2*z*h^2, x^2*y^2*z - x*z^2*h^2, x*y^3*z - y*z^2*h^2, x*y^2*z^2 - z^3*h^2, x^2*y^2*h - x*z*h^3, x*y^3*h - y*z*h^3, x*y^2*z*h - z^2*h^3, x*y^2*h^2 - z*h^4, x^2*y^2 - x*z*h^2, x*y^3 - y*z*h^2, x*y^2*z - z^2*h^2, x*y^2*h - z*h^3, x*y^2 - z*h^2] True False
6 Polynomial Sequence with 35 Polynomials in 4 Variables True False
7 Polynomial Sequence with 71 Polynomials in 4 Variables True True

0 [0] True False
1 [0] True False
2 [0] True False
3 [x*y^2 - z*h^2] True False
4 [x^2*y^2 - x*z*h^2, x*y^3 - y*z*h^2, x*y^2*z - z^2*h^2, x*y^2*h - z*h^3, x*y^2 - z*h^2] True False
5 [x^3*y^2 - x^2*z*h^2, x^2*y^3 - x*y*z*h^2, x*y^4 - y^2*z*h^2, x^2*y^2*z - x*z^2*h^2, x*y^3*z - y*z^2*h^2, x*y^2*z^2 - z^3*h^2, x^2*y^2*h - x*z*h^3, x*y^3*h - y*z*h^3, x*y^2*z*h - z^2*h^3, x

(2, 7, 9, 7, False)

In [349]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = y^2
f2 = y*z + x
test_conjecture_extra([f1,f2],R)

3 

0 [0] True False
1 [0] True False
2 [y^2, y*z + x] False True
3 [x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, y*z^2 + x*z, x^2, x*y, y^2, y*z + x] True True

0 [0] True False
1 [0] True False
2 [y^2, y*z + x] False True
3 [x*y^2, y^3, x*y*z + x^2, y^2*z, y*z^2 + x*z, x*y, y^2, y*z + x] False True
4 [x^2*y^2, x*y^3, y^4, x^2*y*z + x^3, x*y^2*z, y^3*z, x*y*z^2 + x^2*z, y^2*z^2, y*z^3 + x*z^2, x^2*y, x*y^2, y^3, x*y*z, y^2*z, y*z^2 + x*z, x^2, x*y, y^2, y*z + x] True True


(2, 3, 3, 4, False)

In [355]:
R.<x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
f1 = y^2
f2 = y*z + x*h
test_conjecture_extra([f1,f2],R)

3 

0 [0] True False
1 [0] True False
2 [y^2, y*z + x*h] False True
3 [x*y^2, y^3, x*y*z + x^2*h, y^2*z, y*z^2 + x*z*h, x*y*h, y^2*h, y*z*h + x*h^2, y^2, y*z + x*h] False True
4 Polynomial Sequence with 29 Polynomials in 4 Variables True True

0 [0] True False
1 [0] True False
2 [y^2, y*z + x*h] False True
3 [x*y^2, y^3, x*y*z + x^2*h, y^2*z, y*z^2 + x*z*h, x*y*h, y^2*h, y*z*h + x*h^2, y^2, y*z + x*h] False True
4 Polynomial Sequence with 29 Polynomials in 4 Variables True True


(2, 4, 3, 4, False)

In [1]:
singular()

TypeError: Singular.__call__() missing 1 required positional argument: 'x'